# RDFLib Tutorial 

- 작성자: 박하람
- 작성일자: 2022. 09. 05
- 수정일자: 2023. 12. 04
- 내용: 파이썬 라이브러리 RDFLib를 활용해 데이터를 RDF 형식으로 변환하기
- 참고: [RDFLib를 사용해 데이터세트의 메타데이터를 DCAT으로 표현하기 1](https://www.blog.harampark.com/blog/rdflib-tutorial-dcat-1), [RDFLib를 사용해 데이터세트의 메타데이터를 DCAT으로 표현하기 2](https://www.blog.harampark.com/blog/rdflib-tutorial-dcat-2)

### 데이터세트의 메타데이터

- 공공데이터포털의 파일데이터 중 가장 활용도가 높은 데이터세트는 '한국고전번역원_한국문집총간_월고집(月皐集)'으로, '파일데이터 정보'에서 해당 데이터세트에 대한 메타데이터를 제공함
    - 예: 파일데이터명, 분류체계, 제공기관, 관리부서명, 관리부서 전화번호, 업데이트 주기

![월고집 데이터세트의 메타데이터](image/public-dataset-metadata.png)

- 출처: https://www.data.go.kr/data/3074298/fileData.do

### DCAT (Data Catalog Vocabulary)
- 분산된 웹 환경에서 개방된 데이터 카탈로그를 기술하기 위한 RDF 어휘로, 공공데이터 분야에서 데이터세트, 데이터 서비스의 관리와 상호운용을 위해 광범위하게 사용되고 있음
    - `dcat:Catalog`: 자원 (resource)에 대한 메타데이트를 표현하기 위한 클래스로, 데이터세트나 데이터 서비스가 포함된 집합을 기술함
    - `dcat:Dataset`: 데이터세트의 일반적인 특징을 기술하기 위한 클래스 
    - `dcat:DataService`: 데이터를 제공하는 서비스를 기술하기 위한 클래스 
    - `dcat:Distribution`: 데이터세트의 서로 다른 표현 형식 (예: JSON, CSV, XSLX)을 기술하기 위한 클래스

- 명세: https://www.w3.org/TR/vocab-dcat-2/


### 0. 모듈 설치와 불러오기

In [2]:
# rdflib 모듈을 설치하기 위한 코드
!pip install rdflib

In [3]:
# 필요한 모듈 불러오기
import numpy as np
import pandas as pd 
from tqdm import tqdm

### 1. 데이터 불러오기

- 원데이터 출처: [공공데이터포털의 공공데이터 11월 개방현황](https://www.data.go.kr/bbs/ntc/selectNotice.do?pageIndex=1&originId=NOTICE_0000000003409&atchFileId=FILE_000000002847531&nttApiYn=N&searchCondition2=2&searchKeyword1=)
    - 공공데이터포털은 해당 포털이 개방하고 있는 데이터세트의 목록을 제공하고 있음

In [4]:
data = pd.read_csv("data/개방목록현황_11월.csv", encoding="utf-8", dtype=str)
data = data.replace({np.nan : None})
data.head()

,목록키,목록유형,목록명,목록설명,조회수,분류체계,기관코드,기관명,국가중점여부,표준데이터여부,목록 등록일,목록 수정일,목록 URL,다운로드_활용건수,키워드
0,15125221,FILE,경기도 가평군_주정차위반단속위치현황,"가평군의 주정차 위반 단속 위치 현황입니다. 집계년도, 시군명, 관리기관명, 단속일...",(NULL),교통및물류 - 도로,4160000,경기도 가평군,N,N,2023-11-30,2023-11-30,https://www.data.go.kr/data/15125221/fileData.do,0,"주정차단속,단속장소,불법주차"
1,15125222,FILE,한국자활복지개발원_자활근로 참여자 최초보장구분별 현황,"시도별, 최초보장구분별 누적 자활근로 참여자 수<br/> - 최초보장구분 : 일반...",(NULL),사회복지 - 취약계층지원,B554042,한국자활복지개발원,N,N,2023-11-30,2023-11-30,https://www.data.go.kr/data/15125222/fileData.do,0,"자활,자활참여자,최초보장구분"
2,15125223,FILE,한국자활복지개발원_자활근로참여자 연령별 현황,"시도별, 연령별 누적 자활근로 참여자 수<br/> - 연령구분 : 19세이하, 2...",1,사회복지 - 취약계층지원,B554042,한국자활복지개발원,N,N,2023-11-30,2023-11-30,https://www.data.go.kr/data/15125223/fileData.do,0,"자활근로,자활근로 참여자,연령"
3,15125228,FILE,과학기술정보통신부_우주산업 위성활용 부문 매출 및 수출입액,"공공데이터포털에 기 제공되고 있는 우주산업실태조사 내 위성활용 부문 매출 및 수입,...",4,과학기술 - 과학기술연구,1721000,과학기술정보통신부,N,N,2023-11-30,2023-11-30,https://www.data.go.kr/data/15125228/fileData.do,1,"우주산업,우주산업실태조사,위성활용"
4,15125240,FILE,한국자활복지개발원_자활근로 참여자 사업유형별 현황,"시도별, 사업유형별 누적 자활근로 참여자 수<br/> - 유형 : 시장진입형사업단...",40,사회복지 - 취약계층지원,B554042,한국자활복지개발원,N,N,2023-11-30,2023-11-30,https://www.data.go.kr/data/15125240/fileData.do,0,"자활,자활근로,사업유형"


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77419 entries, 0 to 77418
Data columns (total 15 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   목록키        77419 non-null  object
 1   목록유형       77419 non-null  object
 2   목록명        77419 non-null  object
 3   목록설명       77419 non-null  object
 4   조회수        77419 non-null  object
 5   분류체계       77419 non-null  object
 6   기관코드       77419 non-null  object
 7   기관명        77419 non-null  object
 8   국가중점여부     77419 non-null  object
 9   표준데이터여부    77419 non-null  object
 10  목록 등록일     77419 non-null  object
 11  목록 수정일     77419 non-null  object
 12  목록 URL     77419 non-null  object
 13  다운로드_활용건수  77419 non-null  object
 14  키워드        77419 non-null  object
dtypes: object(15)
memory usage: 8.9+ MB


### 2. 데이터 파악하기

In [6]:
print(f"데이터의 행과 열은 {data.shape}입니다.")
print(f"데이터의 컬럼명은 {data.columns}입니다.")

데이터의 행과 열은 (77419, 15)입니다.
데이터의 컬럼명은 Index(['목록키', '목록유형', '목록명', '목록설명', '조회수', '분류체계', '기관코드', '기관명', '국가중점여부',
       '표준데이터여부', '목록 등록일', '목록 수정일', '목록 URL', '다운로드_활용건수', '키워드'],
      dtype='object')입니다.


In [7]:
print(f"고유한 목록키의 개수는 {len(data['목록키'].unique())}개입니다.")

고유한 목록키의 개수는 77419개입니다.


In [8]:
data[data.columns] = data.apply(lambda x: x.str.strip())

### 3. 데이터 변환하기

- [RDFLib Docs](https://rdflib.readthedocs.io/en/stable/gettingstarted.html)

In [9]:
# URI 생성에 필요한 함수 불러오기
from rdflib import Namespace, Literal, URIRef
# 데이터 변환 후 담을 저장소
from rdflib.graph import Graph
# rdflib에 내장된 네임스페이스 불러오기
from rdflib.namespace import CSVW, DC, DCAT, DCTERMS, DOAP, FOAF, ODRL2, ORG, OWL, \
                           PROF, PROV, RDF, RDFS, SDO, SH, SKOS, SOSA, SSN, TIME, \
                           VOID, XMLNS, XSD

In [14]:
%%time

# dcat 표현을 위한 base uri
dcat_id = "http://data.test.com/id/dcat/"

# generate Graph()
g = Graph()
# 어휘와 접두어를 매핑해주는 역할
g.bind("dcat", DCAT)
g.bind("dct", DCTERMS)

for idx, row in tqdm(data[:10].iterrows(), total=data.shape[0]):
    # base id 
    # example: http://data.test.com/id/dcat/목록키
    ds_uri = URIRef(dcat_id + row["목록키"])

    # define uri type
    g.add((ds_uri, RDF.type, DCAT.Dataset))

    # dataset metadata
    g.add((ds_uri, DCTERMS.title, Literal(row["목록명"], datatype=XSD.string)))
    g.add((ds_uri, DCTERMS.issued, Literal(row["목록 등록일"], datatype=XSD.date)))
    g.add((ds_uri, DCTERMS.modified, Literal(row["목록 수정일"], datatype=XSD.date)))
    g.add((ds_uri, DCAT.accessURL, Literal(row["목록 URL"], datatype=XSD.anyURI)))

    # 실습1: 목록 설명을 DCTERMS의 description으로 표현해보기

    # 실습2: 키워드를 DCAT의 속성 keyword로 표현해보기 (단, 키워드는 쉼표 기준으로 분절되어 DCAT.keyword로 표현되어야 함)
    # 예: <http://data.test.com/id/dcat/15118572> dcat:keyword "65세"^^xsd:string, "노인"^^xsd:string, "인구"^^xsd:string .
    # 힌트: print(row["키워드"])

# the number of triples
print(f"총 {len(g)} 개의 트리플이 있습니다.")

# print triples
# print("--- printing raw triples ---")
# for s, p, o in g:
#     print((s, p, o))

# import pprint
# for stmt in g:
#     pprint.pprint(stmt)

# save as ttl
g.serialize(destination=f"data/data-go-kr-metadata-dcat.ttl", format="ttl")

  0%|          | 10/77419 [00:00<01:07, 1138.58it/s]

총 50 개의 트리플이 있습니다.
--- printing raw triples ---
(rdflib.term.URIRef('http://data.test.com/id/dcat/15125221'), rdflib.term.URIRef('http://purl.org/dc/terms/modified'), rdflib.term.Literal('2023-11-30', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#date')))
(rdflib.term.URIRef('http://data.test.com/id/dcat/15125221'), rdflib.term.URIRef('http://purl.org/dc/terms/title'), rdflib.term.Literal('경기도 가평군_주정차위반단속위치현황', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))
(rdflib.term.URIRef('http://data.test.com/id/dcat/15125103'), rdflib.term.URIRef('http://purl.org/dc/terms/issued'), rdflib.term.Literal('2023-11-23', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#date')))
(rdflib.term.URIRef('http://data.test.com/id/dcat/15125207'), rdflib.term.URIRef('http://purl.org/dc/terms/modified'), rdflib.term.Literal('2023-11-30', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#date')))
(rdflib.term.URIRef('http://data.test.com/id/dcat/

<Graph identifier=Na0e271aaabf24a63ac18d52df803c36a (<class 'rdflib.graph.Graph'>)>

### 4. SPARQL 질의하기

In [11]:
# 데이터세트 개수 구하기
query = """
SELECT (COUNT(DISTINCT ?dataset) AS ?dataset_count)
WHERE { 
    ?dataset a dcat:Dataset .
}
"""
result = g.query(query)
for row in result:
    print(f"고유한 데이터세트의 개수는 {row.dataset_count}개입니다.")

고유한 데이터세트의 개수는 10개입니다.


In [12]:
# 데이터세트의 메타데이터 구하기
query = """
SELECT DISTINCT ?dataset ?title ?issued ?modified ?accessURL
WHERE { 
    ?dataset a dcat:Dataset ;
        dct:title ?title ;
        dct:issued ?issued ;
        dct:modified ?modified ;
        dcat:accessURL ?accessURL .
} LIMIT 10
"""
result = g.query(query)
for row in result:
    print(row.dataset, row.title, row.issued, row.modified, row.accessURL)

http://data.test.com/id/dcat/15125221 경기도 가평군_주정차위반단속위치현황 2023-11-30 2023-11-30 https://www.data.go.kr/data/15125221/fileData.do
http://data.test.com/id/dcat/15125222 한국자활복지개발원_자활근로 참여자 최초보장구분별 현황 2023-11-30 2023-11-30 https://www.data.go.kr/data/15125222/fileData.do
http://data.test.com/id/dcat/15125223 한국자활복지개발원_자활근로참여자 연령별 현황 2023-11-30 2023-11-30 https://www.data.go.kr/data/15125223/fileData.do
http://data.test.com/id/dcat/15125228 과학기술정보통신부_우주산업 위성활용 부문 매출 및 수출입액 2023-11-30 2023-11-30 https://www.data.go.kr/data/15125228/fileData.do
http://data.test.com/id/dcat/15125240 한국자활복지개발원_자활근로 참여자 사업유형별 현황 2023-11-30 2023-11-30 https://www.data.go.kr/data/15125240/fileData.do
http://data.test.com/id/dcat/15125207 한국철도공사_서울역 KTX 및 무궁화 승하차 인원 2023-11-29 2023-11-30 https://www.data.go.kr/data/15125207/fileData.do
http://data.test.com/id/dcat/15125190 서울특별시_시 자치구 실외흡연실 설치 현황 2023-11-28 2023-11-30 https://www.data.go.kr/data/15125190/fileData.do
http://data.test.com/id/dcat/15125138 광주광역시 광산구_행